<a href="https://colab.research.google.com/github/RebeccaKessler/Otter_Group_Project/blob/main/fertilizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libaries

import numpy as np
import pandas as pd
import os
import re
import plotly.express as px
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew

# Overview of dataframe


In [ ]:
#importing dataset on fertilizer
df_f = pd.read_csv('fertilizer.csv')

In [ ]:
df_f.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,RFN,Fertilizers by Nutrient,4,Afghanistan,5159,Use per area of cropland,3102,Nutrient nitrogen N (total),1961,1961,kg/ha,0.13,E,Estimated value,NaN
1,RFN,Fertilizers by Nutrient,4,Afghanistan,5159,Use per area of cropland,3102,Nutrient nitrogen N (total),1962,1962,kg/ha,0.13,E,Estimated value,NaN
2,RFN,Fertilizers by Nutrient,4,Afghanistan,5159,Use per area of cropland,3102,Nutrient nitrogen N (total),1963,1963,kg/ha,0.13,E,Estimated value,NaN
3,RFN,Fertilizers by Nutrient,4,Afghanistan,5159,Use per area of cropland,3102,Nutrient nitrogen N (total),1964,1964,kg/ha,0.13,E,Estimated value,NaN
4,RFN,Fertilizers by Nutrient,4,Afghanistan,5159,Use per area of cropland,3102,Nutrient nitrogen N (total),1965,1965,kg/ha,0.13,E,Estimated value,NaN


In [ ]:
#displaying all countries
df_f["Area"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium',
       'Belgium-Luxembourg', 'Belize', 'Benin', 'Bermuda', 'Bhutan',
       'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Canada', 'Cayman Islands', 'Central African Republic', 'Chad',
       'Chile', 'China', 'China, Hong Kong SAR', 'China, mainland',
       'China, Taiwan Province of', 'Colombia', 'Comoros', 'Congo',
       'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba',
       'Cyprus', 'Czechia', 'Czechoslovakia',
       "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican R

In [ ]:
#renaming turkey
df_f['Area'] = df_f['Area'].str.replace('Türkiye', 'Turkey')

In [ ]:
df_f_subset = df_f.loc[:,['Area','Value','Year']]

df_f_subset

,Area,Value,Year
0,Afghanistan,0.13,1961
1,Afghanistan,0.13,1962
2,Afghanistan,0.13,1963
3,Afghanistan,0.13,1964
4,Afghanistan,0.13,1965
...,...,...,...
29713,Zimbabwe,9.76,2017
29714,Zimbabwe,9.00,2018
29715,Zimbabwe,9.00,2019
29716,Zimbabwe,4.07,2020


In [ ]:
#dealing with missing values
df_f_subset.isna().sum()

Area     0
Value    0
Year     0
dtype: int64

In [ ]:
len(list(df_f_subset["Area"].unique()))

217

In [ ]:
df_f_subset = df_f_subset[df_f_subset['Year'] >= 1970]
df_f_subset = df_f_subset[df_f_subset['Year'] <= 2020]
df_f_subset

,Area,Value,Year
9,Afghanistan,2.00,1970
10,Afghanistan,1.84,1971
11,Afghanistan,1.99,1972
12,Afghanistan,2.20,1973
13,Afghanistan,3.01,1974
...,...,...,...
29712,Zimbabwe,8.10,2016
29713,Zimbabwe,9.76,2017
29714,Zimbabwe,9.00,2018
29715,Zimbabwe,9.00,2019


In [ ]:
df_f_subset = df_f_subset.groupby(['Area', 'Year']).agg({'Value': 'sum'}).reset_index()
df_f_subset

,Area,Year,Value
0,Afghanistan,1970,2.42
1,Afghanistan,1971,2.55
2,Afghanistan,1972,3.62
3,Afghanistan,1973,3.06
4,Afghanistan,1974,4.21
...,...,...,...
8832,Zimbabwe,2016,31.78
8833,Zimbabwe,2017,35.71
8834,Zimbabwe,2018,32.39
8835,Zimbabwe,2019,32.39


# Regrouping Countries

In [ ]:
#check until which year we have data on the USSR
df_f_ussr = df_f_subset.loc[df_f_subset["Area"] == 'USSR']
df_f_ussr

,Area,Year,Value
8124,USSR,1970,44.26
8125,USSR,1971,48.74
8126,USSR,1972,53.14
8127,USSR,1973,57.94
8128,USSR,1974,63.29
8129,USSR,1975,74.19
8130,USSR,1976,76.27
8131,USSR,1977,77.50
8132,USSR,1978,79.36
8133,USSR,1979,75.01


In [ ]:
# check from which year we have individual values for the 'former' USSR
df_f_subset.loc[df_f_subset["Area"] == 'Russian Federation']

,Area,Year,Value
6632,Russian Federation,1992,41.21
6633,Russian Federation,1993,29.32
6634,Russian Federation,1994,11.57
6635,Russian Federation,1995,13.52
6636,Russian Federation,1996,12.35
6637,Russian Federation,1997,11.97
6638,Russian Federation,1998,9.88
6639,Russian Federation,1999,10.73
6640,Russian Federation,2000,11.25
6641,Russian Federation,2001,12.75


In [ ]:
#creating subset for 'former' USSR

df_f_formerussr = df_f_subset.loc[df_f_subset['Area'].isin(['Republic of Moldova', 'Belarus','Estonia', 'Latvia', 'Lithuania', 'Kazakhstan', 'Kyrgyzstan', 'Tajikistan', 'Turkmenistan', 'Uzbekistan', 'Russian Federation', 'Armenia', 'Azerbaijan', 'Georgia', 'Ukraine'])]
df_f_formerussr

,Area,Year,Value
290,Armenia,1992,51.76
291,Armenia,1993,30.43
292,Armenia,1994,14.00
293,Armenia,1995,14.17
294,Armenia,1996,15.81
...,...,...,...
8551,Uzbekistan,2016,231.59
8552,Uzbekistan,2017,231.75
8553,Uzbekistan,2018,198.32
8554,Uzbekistan,2019,247.68


In [ ]:
#getting the years of 'former' russia
formerussr_years = df_f_formerussr.loc[:,['Year']].Year.unique()
formerussr_years

array([1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [ ]:
df_f_formerussr = df_f_formerussr.sort_values(by=['Area', 'Year'])

In [ ]:
#getting the unique countries
df_f_formerussr_unique = df_f_formerussr .Area.unique()
print(df_f_formerussr_unique)

['Armenia' 'Azerbaijan' 'Belarus' 'Estonia' 'Georgia' 'Kazakhstan'
 'Kyrgyzstan' 'Latvia' 'Lithuania' 'Republic of Moldova'
 'Russian Federation' 'Tajikistan' 'Turkmenistan' 'Ukraine' 'Uzbekistan']


In [ ]:
#creating a dictionary with the values of the 'former' ussr for each year

formerussr_yearly_values = {}

for year in formerussr_years:
    # Add a condition to consider only years greater than 1990
    if year > 1990:
        values_for_year = df_f_formerussr[df_f_formerussr['Year'] == year]['Value'].tolist()
        formerussr_yearly_values[year] = values_for_year

# Print the values for years greater than 1990
for year, values in formerussr_yearly_values.items():
    print(f'Year {year}: {values}')

Year 1992: [51.76, 37.23, 222.81, 100.01, 63.78, 13.489999999999998, 23.05, 98.25, 53.26, 61.22, 41.21, 129.94, 112.91, 78.13, 150.39]
Year 1993: [30.43, 26.650000000000002, 120.36, 60.31, 47.78, 9.209999999999999, 19.71, 55.7, 30.29, 52.46, 29.32, 76.15, 96.97, 39.08, 131.22]
Year 1994: [14.0, 19.19, 84.75, 43.370000000000005, 27.51, 3.48, 19.69, 54.769999999999996, 30.04, 52.51, 11.57, 72.16, 71.42999999999999, 33.0, 97.72]
Year 1995: [14.17, 19.16, 81.99000000000001, 31.59, 28.76, 2.73, 19.66, 21.34, 40.56, 52.61, 13.52, 72.99, 72.78, 25.93, 97.87]
Year 1996: [15.81, 8.11, 121.87, 24.75, 30.21, 3.95, 21.75, 23.240000000000002, 40.9, 53.07, 12.35, 67.57, 78.89, 15.35, 91.94]
Year 1997: [16.06, 11.489999999999998, 135.31, 30.9, 34.33, 1.71, 21.740000000000002, 32.08, 47.0, 4.67, 11.97, 55.76, 98.89, 16.49, 180.92]
Year 1998: [19.51, 7.67, 146.57999999999998, 36.03, 32.02, 0.41000000000000003, 20.37, 45.94, 47.67, 3.21, 9.88, 41.56, 42.230000000000004, 15.2, 170.54000000000002]
Year 19

In [ ]:
#define function to calculate 'former' USSR
def weighted_sum(values, weights):
    """
    Compute the weighted sum of values.

    :param values: A list of values to be weighted.
    :param weights: A list of weights corresponding to the values.
    :return: The weighted sum.
    """
    if len(values) != len(weights):
        raise ValueError("Number of values and weights must be the same.")

    # Use a list comprehension to compute the weighted products and sum them up
    weighted_products = [value * weight for value, weight in zip(values, weights)]
    total_weighted_sum = sum(weighted_products)

    return total_weighted_sum

In [ ]:
#exectue function for 'former' ussr
weights = [0.0013,0.0039, 0.0093, 0.0020, 0.0031, 0.1224, 0.0089, 0.0029, 0.0029, 0.0015, 0.7662, 0.0064, 0.0219, 0.0271, 0.0201]
# source wikipedia

weigthed_sum_formerussr = {}
for year in formerussr_yearly_values:
    values = formerussr_yearly_values[year]
    result = weighted_sum(values, weights)
    result = "{:.3f}".format(result)
    result = float(result)
    result_dict = {'Area':'USSR', 'Value': result, 'Year':year}
    weigthed_sum_formerussr[year] = result_dict

weigthed_sum_formerussr

{1992: {'Area': 'USSR', 'Value': 45.089, 'Year': 1992},
 1993: {'Area': 'USSR', 'Value': 31.935, 'Year': 1993},
 1994: {'Area': 'USSR', 'Value': 15.729, 'Year': 1994},
 1995: {'Area': 'USSR', 'Value': 16.865, 'Year': 1995},
 1996: {'Area': 'USSR', 'Value': 16.158, 'Year': 1996},
 1997: {'Area': 'USSR', 'Value': 17.909, 'Year': 1997},
 1998: {'Area': 'USSR', 'Value': 14.698, 'Year': 1998},
 1999: {'Area': 'USSR', 'Value': 14.918, 'Year': 1999},
 2000: {'Area': 'USSR', 'Value': 15.249, 'Year': 2000},
 2001: {'Area': 'USSR', 'Value': 16.67, 'Year': 2001},
 2002: {'Area': 'USSR', 'Value': 15.603, 'Year': 2002},
 2003: {'Area': 'USSR', 'Value': 15.404, 'Year': 2003},
 2004: {'Area': 'USSR', 'Value': 15.885, 'Year': 2004},
 2005: {'Area': 'USSR', 'Value': 16.958, 'Year': 2005},
 2006: {'Area': 'USSR', 'Value': 19.399, 'Year': 2006},
 2007: {'Area': 'USSR', 'Value': 20.858, 'Year': 2007},
 2008: {'Area': 'USSR', 'Value': 22.414, 'Year': 2008},
 2009: {'Area': 'USSR', 'Value': 23.046, 'Year': 

In [ ]:
#convert dictionary to dataframe

df_f_formerussr= pd.DataFrame.from_dict(weigthed_sum_formerussr,
                                         orient = 'index',
                                         columns = ['Area', 'Value', 'Year'])
df_f_formerussr.reset_index(drop=True)

,Area,Value,Year
0,USSR,45.089,1992
1,USSR,31.935,1993
2,USSR,15.729,1994
3,USSR,16.865,1995
4,USSR,16.158,1996
5,USSR,17.909,1997
6,USSR,14.698,1998
7,USSR,14.918,1999
8,USSR,15.249,2000
9,USSR,16.670,2001


In [ ]:
df_f_subset = pd.concat([df_f_subset, df_f_formerussr])
df_f_subset = df_f_subset.reset_index(drop=True)
df_f_subset = df_f_subset.sort_values(['Area', 'Year'])
df_f_subset

,Area,Year,Value
0,Afghanistan,1970,2.42
1,Afghanistan,1971,2.55
2,Afghanistan,1972,3.62
3,Afghanistan,1973,3.06
4,Afghanistan,1974,4.21
...,...,...,...
8832,Zimbabwe,2016,31.78
8833,Zimbabwe,2017,35.71
8834,Zimbabwe,2018,32.39
8835,Zimbabwe,2019,32.39


In [ ]:
df_f_subset.loc[df_f_subset["Area"] == 'USSR']

,Area,Year,Value
8124,USSR,1970,44.260
8125,USSR,1971,48.740
8126,USSR,1972,53.140
8127,USSR,1973,57.940
8128,USSR,1974,63.290
8129,USSR,1975,74.190
8130,USSR,1976,76.270
8131,USSR,1977,77.500
8132,USSR,1978,79.360
8133,USSR,1979,75.010


In [ ]:
#removing the individual countries of former USSR from the datatset
df_f_subset = df_f_subset.loc[~df_f_subset['Area'].isin(['Republic of Moldova', 'Belarus','Estonia', 'Latvia', 'Lithuania', 'Kazakhstan', 'Kyrgyzstan', 'Tajikistan', 'Turkmenistan', 'Uzbekistan', 'Russian Federation', 'Armenia', 'Azerbaijan', 'Georgia', 'Ukraine'])]
df_f_subset

,Area,Year,Value
0,Afghanistan,1970,2.42
1,Afghanistan,1971,2.55
2,Afghanistan,1972,3.62
3,Afghanistan,1973,3.06
4,Afghanistan,1974,4.21
...,...,...,...
8832,Zimbabwe,2016,31.78
8833,Zimbabwe,2017,35.71
8834,Zimbabwe,2018,32.39
8835,Zimbabwe,2019,32.39


In [ ]:
len(list(df_f_subset["Area"].unique()))

202

In [ ]:
#check until which year we have data on Yugoslavia
df_f_yguo = df_f_subset.loc[df_f_subset["Area"] == 'Yugoslav SFR']
df_f_yguo

,Area,Year,Value
8713,Yugoslav SFR,1970,76.95
8714,Yugoslav SFR,1971,81.90
8715,Yugoslav SFR,1972,88.07
8716,Yugoslav SFR,1973,87.30
8717,Yugoslav SFR,1974,83.56
8718,Yugoslav SFR,1975,89.62
8719,Yugoslav SFR,1976,92.20
8720,Yugoslav SFR,1977,100.90
8721,Yugoslav SFR,1978,107.85
8722,Yugoslav SFR,1979,110.18


In [ ]:
#creating a subset of 'former' yugoslavia countries
df_f_formeryugo = df_f_subset.loc[df_f_subset['Area'].isin(['Bosnia and Herzegovina','Croatia','North Macedonia','Serbia and Montenegro','Serbia', 'Montenegro','Kosovo','Slovenia'])]
df_f_formeryugo

,Area,Year,Value
947,Bosnia and Herzegovina,1995,15.39
948,Bosnia and Herzegovina,1996,8.82
949,Bosnia and Herzegovina,1997,8.88
950,Bosnia and Herzegovina,1998,44.34
951,Bosnia and Herzegovina,1999,39.82
...,...,...,...
7283,Slovenia,2016,126.43
7284,Slovenia,2017,125.34
7285,Slovenia,2018,127.34
7286,Slovenia,2019,124.09


In [ ]:
#getting the unique countries
df_f_formeryugo_unique = df_f_formeryugo.Area.unique()
print(df_f_formeryugo_unique)

['Bosnia and Herzegovina' 'Croatia' 'Montenegro' 'North Macedonia'
 'Serbia' 'Serbia and Montenegro' 'Slovenia']


In [ ]:
#subset with montenegro and serbia
df_f_ms = df_f_formeryugo.loc[df_f_formeryugo['Area'].isin(['Serbia', 'Montenegro'])]
df_f_ms

,Area,Year,Value
5265,Montenegro,2006,236.82
5266,Montenegro,2007,197.96
5267,Montenegro,2008,186.25
5268,Montenegro,2009,142.65
5269,Montenegro,2010,179.69
5270,Montenegro,2011,158.00
5271,Montenegro,2012,156.36
5272,Montenegro,2013,184.45
5273,Montenegro,2014,175.94
5274,Montenegro,2015,165.28


In [ ]:
#getting the years of montenegro and serbia
ms_years = df_f_ms.loc[:,['Year']].Year.unique()
ms_years

array([2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2020])

In [ ]:
#creating a dictionary with the values of montenegro and serbia for each year

ms_yearly_values = {}

for year in ms_years:
  values_for_year = df_f_ms[df_f_ms['Year'] == year]['Value'].tolist()
  ms_yearly_values[year] = values_for_year


for year, values in ms_yearly_values.items():
    print(f'Year {year}: {values}')

Year 2006: [236.82, 134.54]
Year 2007: [197.96, 174.22]
Year 2008: [186.25, 126.57000000000001]
Year 2009: [142.65, 158.26]
Year 2010: [179.69, 119.32]
Year 2011: [158.0, 134.72]
Year 2012: [156.36, 192.85]
Year 2013: [184.45, 172.95]
Year 2014: [175.94, 113.19999999999999]
Year 2015: [165.28, 99.72]
Year 2016: [178.25, 129.5]
Year 2017: [147.22, 111.93]
Year 2018: [155.51999999999998, 67.55]
Year 2019: [175.69, 80.65]
Year 2020: [190.61, 148.51]


In [ ]:
#exectue function for montenegro and serbia
weights = [0.1352,0.8648]
# source wikipedia

weigthed_sum_ms = {}
for year in ms_yearly_values:
    values = ms_yearly_values[year]
    result = weighted_sum(values, weights)
    result = "{:.3f}".format(result)
    result = float(result)
    result_dict = {'Area':'Serbia and Montenegro', 'Value': result, 'Year':year}
    weigthed_sum_ms[year] = result_dict

weigthed_sum_ms

{2006: {'Area': 'Serbia and Montenegro', 'Value': 148.368, 'Year': 2006},
 2007: {'Area': 'Serbia and Montenegro', 'Value': 177.43, 'Year': 2007},
 2008: {'Area': 'Serbia and Montenegro', 'Value': 134.639, 'Year': 2008},
 2009: {'Area': 'Serbia and Montenegro', 'Value': 156.15, 'Year': 2009},
 2010: {'Area': 'Serbia and Montenegro', 'Value': 127.482, 'Year': 2010},
 2011: {'Area': 'Serbia and Montenegro', 'Value': 137.867, 'Year': 2011},
 2012: {'Area': 'Serbia and Montenegro', 'Value': 187.917, 'Year': 2012},
 2013: {'Area': 'Serbia and Montenegro', 'Value': 174.505, 'Year': 2013},
 2014: {'Area': 'Serbia and Montenegro', 'Value': 121.682, 'Year': 2014},
 2015: {'Area': 'Serbia and Montenegro', 'Value': 108.584, 'Year': 2015},
 2016: {'Area': 'Serbia and Montenegro', 'Value': 136.091, 'Year': 2016},
 2017: {'Area': 'Serbia and Montenegro', 'Value': 116.701, 'Year': 2017},
 2018: {'Area': 'Serbia and Montenegro', 'Value': 79.444, 'Year': 2018},
 2019: {'Area': 'Serbia and Montenegro', 

In [ ]:
#convert dictionary to dataframe

df_f_ms= pd.DataFrame.from_dict(weigthed_sum_ms,
                                         orient = 'index',
                                         columns = ['Area', 'Value', 'Year'])
df_f_ms = df_f_ms.reset_index(drop=True)
df_f_ms

,Area,Value,Year
0,Serbia and Montenegro,148.368,2006
1,Serbia and Montenegro,177.430,2007
2,Serbia and Montenegro,134.639,2008
3,Serbia and Montenegro,156.150,2009
4,Serbia and Montenegro,127.482,2010
5,Serbia and Montenegro,137.867,2011
6,Serbia and Montenegro,187.917,2012
7,Serbia and Montenegro,174.505,2013
8,Serbia and Montenegro,121.682,2014
9,Serbia and Montenegro,108.584,2015


In [ ]:
#add dataframe to original dataframe

df_f_formeryugo = pd.concat([df_f_ms, df_f_formeryugo])
df_f_formeryugo  = df_f_formeryugo.reset_index(drop=True)
df_f_formeryugo  = df_f_formeryugo.sort_values(['Area', 'Year'])
df_f_formeryugo

,Area,Value,Year
15,Bosnia and Herzegovina,15.39,1995
16,Bosnia and Herzegovina,8.82,1996
17,Bosnia and Herzegovina,8.88,1997
18,Bosnia and Herzegovina,44.34,1998
19,Bosnia and Herzegovina,39.82,1999
...,...,...,...
166,Slovenia,126.43,2016
167,Slovenia,125.34,2017
168,Slovenia,127.34,2018
169,Slovenia,124.09,2019


In [ ]:
#removing serbia and montenegro from the datatset
df_f_formeryugo = df_f_formeryugo.loc[~df_f_formeryugo['Area'].isin(['Serbia', 'Montenegro'])]
df_f_formeryugo

,Area,Value,Year
15,Bosnia and Herzegovina,15.39,1995
16,Bosnia and Herzegovina,8.82,1996
17,Bosnia and Herzegovina,8.88,1997
18,Bosnia and Herzegovina,44.34,1998
19,Bosnia and Herzegovina,39.82,1999
...,...,...,...
166,Slovenia,126.43,2016
167,Slovenia,125.34,2017
168,Slovenia,127.34,2018
169,Slovenia,124.09,2019


In [ ]:
#checking unique countries
df_f_formeryugo_unique = df_f_formeryugo.Area.unique()
print(df_f_formeryugo_unique)

['Bosnia and Herzegovina' 'Croatia' 'North Macedonia'
 'Serbia and Montenegro' 'Slovenia']


In [ ]:
new_row = {'Year': 1991, 'Area': 'Croatia', 'Value': 138.51}
df_f_formeryugo = df_f_formeryugo.append(new_row, ignore_index=True)
df_f_formeryugo = df_f_formeryugo.sort_values(by=['Year', 'Area'])

<ipython-input-36-24873f619ed9>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_formeryugo = df_f_formeryugo.append(new_row, ignore_index=True)


In [ ]:
#filling missing values for relevant countries with good estimates (years before or after)
new_row = {'Year': 1991, 'Area': 'Bosnia and Herzegovina', 'Value': 15.39}
df_f_formeryugo = df_f_formeryugo.append(new_row, ignore_index=True)
new_row = {'Year': 1992, 'Area': 'Bosnia and Herzegovina', 'Value': 15.39}
df_f_formeryugo = df_f_formeryugo.append(new_row, ignore_index=True)
new_row = {'Year': 1993, 'Area': 'Bosnia and Herzegovina', 'Value': 15.39}
df_f_formeryugo = df_f_formeryugo.append(new_row, ignore_index=True)
new_row = {'Year': 1994, 'Area': 'Bosnia and Herzegovina', 'Value': 15.39}
df_f_formeryugo = df_f_formeryugo.append(new_row, ignore_index=True)

df_f_formeryugo = df_f_formeryugo.sort_values(by=['Year', 'Area'])

<ipython-input-37-91f8ca70919e>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_formeryugo = df_f_formeryugo.append(new_row, ignore_index=True)
<ipython-input-37-91f8ca70919e>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_formeryugo = df_f_formeryugo.append(new_row, ignore_index=True)
<ipython-input-37-91f8ca70919e>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_formeryugo = df_f_formeryugo.append(new_row, ignore_index=True)
<ipython-input-37-91f8ca70919e>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_formeryugo = df_f_formeryugo.append(new_row, ignore_index=True)


In [ ]:
new_row = {'Year': 1991, 'Area': 'North Macedonia', 'Value': 11.46}
df_f_formeryugo = df_f_formeryugo.append(new_row, ignore_index=True)
new_row = {'Year': 1992, 'Area': 'North Macedonia', 'Value': 11.46}
df_f_formeryugo = df_f_formeryugo.append(new_row, ignore_index=True)
df_f_formeryugo = df_f_formeryugo.sort_values(by=['Year', 'Area'])

<ipython-input-38-df1c47af0f57>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_formeryugo = df_f_formeryugo.append(new_row, ignore_index=True)
<ipython-input-38-df1c47af0f57>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_formeryugo = df_f_formeryugo.append(new_row, ignore_index=True)


In [ ]:
df_f_formeryugo = df_f_formeryugo.loc[df_f_formeryugo['Year'] > 1991]

In [ ]:
df_f_formeryugo = df_f_formeryugo.sort_values(by=['Area', 'Year'])
df_f_formeryugo

,Area,Value,Year
2,Bosnia and Herzegovina,15.39,1992
6,Bosnia and Herzegovina,15.39,1993
11,Bosnia and Herzegovina,15.39,1994
16,Bosnia and Herzegovina,15.39,1995
21,Bosnia and Herzegovina,8.82,1996
...,...,...,...
125,Slovenia,126.43,2016
130,Slovenia,125.34,2017
135,Slovenia,127.34,2018
140,Slovenia,124.09,2019


In [ ]:
#getting the years of 'former' yugo
formeryugo_years = df_f_formeryugo.loc[:,['Year']].Year.unique()
formeryugo_years

array([1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [ ]:
#creating a dictionary with the values of the 'former' ussr for each year

formeryugo_yearly_values = {}

for year in formeryugo_years:
  values_for_year = df_f_formeryugo[df_f_formeryugo['Year'] == year]['Value'].tolist()
  formeryugo_yearly_values[year] = values_for_year


for year, values in formeryugo_yearly_values.items():
  print(f'Year {year}: {values}')

Year 1992: [15.39, 138.51, 11.46, 24.17, 167.66]
Year 1993: [15.39, 117.47, 11.46, 17.73, 145.91]
Year 1994: [15.39, 158.07, 89.14, 20.57, 234.04]
Year 1995: [15.39, 146.8, 56.75, 35.01, 205.26]
Year 1996: [8.82, 182.08, 65.96000000000001, 52.82, 182.19]
Year 1997: [8.879999999999999, 248.96, 72.18, 60.120000000000005, 230.52]
Year 1998: [44.34, 144.82, 61.739999999999995, 50.16, 241.3]
Year 1999: [39.82, 162.24, 68.72, 51.72, 249.27]
Year 2000: [37.99, 244.79, 66.61, 51.09, 236.69]
Year 2001: [29.44, 190.26, 36.44, 66.69, 227.57999999999998]
Year 2002: [29.82, 237.6, 34.75, 86.91, 217.96]
Year 2003: [16.14, 270.57, 36.74, 87.60000000000001, 218.12]
Year 2004: [53.14, 288.72, 48.44, 135.53, 196.35]
Year 2005: [52.72, 271.79, 57.05, 117.24, 180.95]
Year 2006: [38.59, 350.55, 52.89, 148.368, 176.73000000000002]
Year 2007: [41.260000000000005, 374.87, 61.94, 177.43, 179.53]
Year 2008: [54.33, 450.24, 51.76, 134.639, 140.1]
Year 2009: [66.04, 149.52, 52.34, 156.15, 114.89999999999999]
Year

In [ ]:
#exectue function for 'former' yugo
weights = [0.1919, 0.2121, 0.1005, 0.4237, 0.0760]
# source wikipedia

weigthed_sum_formeryugo = {}
for year in formeryugo_yearly_values:
    values = formeryugo_yearly_values[year]
    result = weighted_sum(values, weights)
    result = "{:.3f}".format(result)
    result = float(result)
    result_dict = {'Area':'Yugoslav SFR', 'Value': result, 'Year':year}
    weigthed_sum_formeryugo[year] = result_dict

weigthed_sum_formeryugo

{1992: {'Area': 'Yugoslav SFR', 'Value': 56.466, 'Year': 1992},
 1993: {'Area': 'Yugoslav SFR', 'Value': 47.622, 'Year': 1993},
 1994: {'Area': 'Yugoslav SFR', 'Value': 71.941, 'Year': 1994},
 1995: {'Area': 'Yugoslav SFR', 'Value': 70.226, 'Year': 1995},
 1996: {'Area': 'Yugoslav SFR', 'Value': 83.167, 'Year': 1996},
 1997: {'Area': 'Yugoslav SFR', 'Value': 104.755, 'Year': 1997},
 1998: {'Area': 'Yugoslav SFR', 'Value': 85.022, 'Year': 1998},
 1999: {'Area': 'Yugoslav SFR', 'Value': 89.817, 'Year': 1999},
 2000: {'Area': 'Yugoslav SFR', 'Value': 105.54, 'Year': 2000},
 2001: {'Area': 'Yugoslav SFR', 'Value': 95.219, 'Year': 2001},
 2002: {'Area': 'Yugoslav SFR', 'Value': 112.999, 'Year': 2002},
 2003: {'Area': 'Yugoslav SFR', 'Value': 117.871, 'Year': 2003},
 2004: {'Area': 'Yugoslav SFR', 'Value': 148.65, 'Year': 2004},
 2005: {'Area': 'Yugoslav SFR', 'Value': 136.924, 'Year': 2005},
 2006: {'Area': 'Yugoslav SFR', 'Value': 163.368, 'Year': 2006},
 2007: {'Area': 'Yugoslav SFR', 'Va

In [ ]:
#convert dictionary to dataframe

df_f_formeryugo= pd.DataFrame.from_dict(weigthed_sum_formeryugo,
                                         orient = 'index',
                                         columns = ['Area', 'Value', 'Year'])
df_f_formeryugo.reset_index(drop=True)

,Area,Value,Year
0,Yugoslav SFR,56.466,1992
1,Yugoslav SFR,47.622,1993
2,Yugoslav SFR,71.941,1994
3,Yugoslav SFR,70.226,1995
4,Yugoslav SFR,83.167,1996
5,Yugoslav SFR,104.755,1997
6,Yugoslav SFR,85.022,1998
7,Yugoslav SFR,89.817,1999
8,Yugoslav SFR,105.540,2000
9,Yugoslav SFR,95.219,2001


In [ ]:
#add dataframe to original dataframe

df_f_subset = pd.concat([df_f_subset, df_f_formeryugo])
df_f_subset = df_f_subset.reset_index(drop=True)
df_f_subset = df_f_subset.sort_values(['Area', 'Year'])
df_f_subset

,Area,Year,Value
0,Afghanistan,1970,2.42
1,Afghanistan,1971,2.55
2,Afghanistan,1972,3.62
3,Afghanistan,1973,3.06
4,Afghanistan,1974,4.21
...,...,...,...
8426,Zimbabwe,2016,31.78
8427,Zimbabwe,2017,35.71
8428,Zimbabwe,2018,32.39
8429,Zimbabwe,2019,32.39


In [ ]:
#removing the individual countries of former yugoslavia from the datatset

df_f_subset = df_f_subset.loc[~df_f_subset['Area'].isin(['Bosnia and Herzegovina', 'Croatia', 'North Macedonia','Serbia and Montenegro', 'Serbia', 'Montenegro', 'Slovenia'])]
df_f_subset

,Area,Year,Value
0,Afghanistan,1970,2.42
1,Afghanistan,1971,2.55
2,Afghanistan,1972,3.62
3,Afghanistan,1973,3.06
4,Afghanistan,1974,4.21
...,...,...,...
8426,Zimbabwe,2016,31.78
8427,Zimbabwe,2017,35.71
8428,Zimbabwe,2018,32.39
8429,Zimbabwe,2019,32.39


In [ ]:
#sample size
len(list(df_f_subset["Area"].unique()))

195

In [ ]:
#dealing with czechosloviakia

df_f_czechoslovakia = df_f_subset.loc[df_f_subset['Area'].isin(['Czechia','Slovakia'])]
df_f_czechoslovakia

,Area,Year,Value
1968,Czechia,1993,89.86
1969,Czechia,1994,99.21
1970,Czechia,1995,102.94
1971,Czechia,1996,103.99
1972,Czechia,1997,94.40
1973,Czechia,1998,89.87
1974,Czechia,1999,80.69
1975,Czechia,2000,99.64
1976,Czechia,2001,119.17
1977,Czechia,2002,93.75


In [ ]:
#dealing with czechosloviakia
czechoslovakia_years = df_f_czechoslovakia.loc[:,['Year']].Year.unique()
czechoslovakia_years

array([1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020])

In [ ]:
#dealing with czechosloviakia
czechoslovakia_yearly_values = {}

for year in czechoslovakia_years:
  values_for_year = df_f_czechoslovakia[df_f_czechoslovakia['Year'] == year]['Value'].tolist()
  czechoslovakia_yearly_values[year] = values_for_year


for year, values in czechoslovakia_yearly_values.items():
  print(f'Year {year}: {values}')

Year 1993: [89.86, 59.800000000000004]
Year 1994: [99.21000000000001, 61.489999999999995]
Year 1995: [102.94, 66.95]
Year 1996: [103.99, 74.13]
Year 1997: [94.4, 67.16]
Year 1998: [89.87, 75.49000000000001]
Year 1999: [80.69, 56.1]
Year 2000: [99.64, 75.01]
Year 2001: [119.16999999999999, 79.77]
Year 2002: [93.75, 81.49]
Year 2003: [105.66000000000001, 77.42]
Year 2004: [100.89999999999999, 82.08]
Year 2005: [104.39, 79.03]
Year 2006: [112.77999999999999, 90.14]
Year 2007: [120.1, 88.35]
Year 2008: [105.44, 73.86]
Year 2009: [107.4, 76.97]
Year 2010: [117.44000000000001, 83.54]
Year 2011: [124.2, 94.56]
Year 2012: [157.54000000000002, 105.39]
Year 2013: [158.57, 107.67999999999999]
Year 2014: [159.70999999999998, 114.88]
Year 2015: [188.82999999999998, 111.51]
Year 2016: [192.67, 124.10000000000001]
Year 2017: [179.05, 120.08000000000001]
Year 2018: [170.99, 127.69999999999999]
Year 2019: [162.58999999999997, 127.49]
Year 2020: [141.35999999999999, 128.1]


In [ ]:
#dealing with czechosloviakia
weights = [0.7229, 0.2771, ]
# source wikipedia

weigthed_sum_czechoslovakia = {}
for year in czechoslovakia_yearly_values:
    values = czechoslovakia_yearly_values[year]
    result = weighted_sum(values, weights)
    result = "{:.3f}".format(result)
    result = float(result)
    result_dict = {'Area':'Czechoslovakia', 'Value': result, 'Year':year}
    weigthed_sum_czechoslovakia[year] = result_dict

weigthed_sum_czechoslovakia

{1993: {'Area': 'Czechoslovakia', 'Value': 81.53, 'Year': 1993},
 1994: {'Area': 'Czechoslovakia', 'Value': 88.758, 'Year': 1994},
 1995: {'Area': 'Czechoslovakia', 'Value': 92.967, 'Year': 1995},
 1996: {'Area': 'Czechoslovakia', 'Value': 95.716, 'Year': 1996},
 1997: {'Area': 'Czechoslovakia', 'Value': 86.852, 'Year': 1997},
 1998: {'Area': 'Czechoslovakia', 'Value': 85.885, 'Year': 1998},
 1999: {'Area': 'Czechoslovakia', 'Value': 73.876, 'Year': 1999},
 2000: {'Area': 'Czechoslovakia', 'Value': 92.815, 'Year': 2000},
 2001: {'Area': 'Czechoslovakia', 'Value': 108.252, 'Year': 2001},
 2002: {'Area': 'Czechoslovakia', 'Value': 90.353, 'Year': 2002},
 2003: {'Area': 'Czechoslovakia', 'Value': 97.835, 'Year': 2003},
 2004: {'Area': 'Czechoslovakia', 'Value': 95.685, 'Year': 2004},
 2005: {'Area': 'Czechoslovakia', 'Value': 97.363, 'Year': 2005},
 2006: {'Area': 'Czechoslovakia', 'Value': 106.506, 'Year': 2006},
 2007: {'Area': 'Czechoslovakia', 'Value': 111.302, 'Year': 2007},
 2008: {

In [ ]:
#dealing with czechosloviakia
df_f_czechoslovakia= pd.DataFrame.from_dict(weigthed_sum_czechoslovakia,
                                         orient = 'index',
                                         columns = ['Area', 'Value', 'Year'])
df_f_czechoslovakia.reset_index(drop=True)

,Area,Value,Year
0,Czechoslovakia,81.530,1993
1,Czechoslovakia,88.758,1994
2,Czechoslovakia,92.967,1995
3,Czechoslovakia,95.716,1996
4,Czechoslovakia,86.852,1997
5,Czechoslovakia,85.885,1998
6,Czechoslovakia,73.876,1999
7,Czechoslovakia,92.815,2000
8,Czechoslovakia,108.252,2001
9,Czechoslovakia,90.353,2002


In [ ]:
#dealing with czechosloviakia
df_f_subset = pd.concat([df_f_subset, df_f_czechoslovakia])
df_f_subset = df_f_subset.reset_index(drop=True)
df_f_subset = df_f_subset.sort_values(['Area', 'Year'])
df_f_subset

,Area,Year,Value
0,Afghanistan,1970,2.42
1,Afghanistan,1971,2.55
2,Afghanistan,1972,3.62
3,Afghanistan,1973,3.06
4,Afghanistan,1974,4.21
...,...,...,...
8299,Zimbabwe,2016,31.78
8300,Zimbabwe,2017,35.71
8301,Zimbabwe,2018,32.39
8302,Zimbabwe,2019,32.39


In [ ]:
#dealing with czechosloviakia
df_f_subset = df_f_subset.loc[~df_f_subset['Area'].isin(['Czechia','Slovakia'])]
df_f_subset

,Area,Year,Value
0,Afghanistan,1970,2.42
1,Afghanistan,1971,2.55
2,Afghanistan,1972,3.62
3,Afghanistan,1973,3.06
4,Afghanistan,1974,4.21
...,...,...,...
8299,Zimbabwe,2016,31.78
8300,Zimbabwe,2017,35.71
8301,Zimbabwe,2018,32.39
8302,Zimbabwe,2019,32.39


In [ ]:
df_f_subset.loc[df_f_subset["Area"] == 'Czechoslovakia']

,Area,Year,Value
1941,Czechoslovakia,1970,240.400
1942,Czechoslovakia,1971,257.590
1943,Czechoslovakia,1972,266.270
1944,Czechoslovakia,1973,264.680
1945,Czechoslovakia,1974,292.780
1946,Czechoslovakia,1975,320.400
1947,Czechoslovakia,1976,318.570
1948,Czechoslovakia,1977,310.710
1949,Czechoslovakia,1978,333.400
1950,Czechoslovakia,1979,334.680


In [ ]:
for y in list(range(1990,2022,1)):
  value = df_f_subset.loc[(df_f_subset.Area == "Ethiopia") & (df_f_subset.Year == y),"Value"].sum()
  new_row = {"Area": "Ethiopia PDR","Value": value, "Year": y}
  df_f_subset = df_f_subset.append(new_row,ignore_index=True)
df_f_subset = df_f_subset.sort_values(by=["Area","Year"])
df_f_subset = df_f_subset.loc[~(df_f_subset.Area=="Ethiopia")]

<ipython-input-56-503cdb7d17e4>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_subset = df_f_subset.append(new_row,ignore_index=True)
<ipython-input-56-503cdb7d17e4>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_subset = df_f_subset.append(new_row,ignore_index=True)
<ipython-input-56-503cdb7d17e4>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_subset = df_f_subset.append(new_row,ignore_index=True)
<ipython-input-56-503cdb7d17e4>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_subset = df_f_subset.append(new_row,ignore_index=True)
<ipython-input-56-503cdb7d17e4>:4: FutureWarning: The frame.append method is dep

In [ ]:
df_f_subset.loc[df_f_subset["Area"] == 'Belgium']

,Area,Year,Value
592,Belgium,2002,318.16
593,Belgium,2003,331.02
594,Belgium,2004,328.94
595,Belgium,2005,324.32
596,Belgium,2006,317.94
597,Belgium,2007,326.94
598,Belgium,2008,218.96
599,Belgium,2009,274.95
600,Belgium,2010,316.70
601,Belgium,2011,309.66


In [ ]:
df_f_subset.loc[df_f_subset["Area"] == 'Luxembourg']

,Area,Year,Value
4396,Luxembourg,2002,118.32
4397,Luxembourg,2003,118.32
4398,Luxembourg,2004,146.54
4399,Luxembourg,2005,137.50
4400,Luxembourg,2006,126.46
4401,Luxembourg,2007,119.54
4402,Luxembourg,2008,104.79
4403,Luxembourg,2009,101.84
4404,Luxembourg,2010,108.89
4405,Luxembourg,2011,114.03


In [ ]:
df_f_subset.loc[df_f_subset["Area"] == 'Belgium-Luxembourg']

,Area,Year,Value
611,Belgium-Luxembourg,1970,497.72
612,Belgium-Luxembourg,1971,509.71
613,Belgium-Luxembourg,1972,511.82
614,Belgium-Luxembourg,1973,528.93
615,Belgium-Luxembourg,1974,520.95
616,Belgium-Luxembourg,1975,451.44
617,Belgium-Luxembourg,1976,463.11
618,Belgium-Luxembourg,1977,457.56
619,Belgium-Luxembourg,1978,477.81
620,Belgium-Luxembourg,1979,455.68


In [ ]:
#merging Belgium-Luxembourg and Belgium

for y in list(range(2000,2022,1)):
  value = df_f_subset.loc[(df_f_subset.Area == "Belgium") & (df_f_subset.Year == y),"Value"].sum()
  new_row = {"Area": "Belgium-Luxembourg","Value": value, "Year": y}
  df_f_subset = df_f_subset.append(new_row,ignore_index=True)
df_f_subset = df_f_subset.sort_values(by=["Area","Year"])
df_f_subset = df_f_subset.loc[~(df_f_subset.Area=="Belgium")]
df_f_subset = df_f_subset.loc[~(df_f_subset.Area=="Luxembourg")]

<ipython-input-60-2bc8bcd0e8ba>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_subset = df_f_subset.append(new_row,ignore_index=True)
<ipython-input-60-2bc8bcd0e8ba>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_subset = df_f_subset.append(new_row,ignore_index=True)
<ipython-input-60-2bc8bcd0e8ba>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_subset = df_f_subset.append(new_row,ignore_index=True)
<ipython-input-60-2bc8bcd0e8ba>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_f_subset = df_f_subset.append(new_row,ignore_index=True)
<ipython-input-60-2bc8bcd0e8ba>:6: FutureWarning: The frame.append method is dep

In [ ]:
df_f_subset.loc[df_f_subset["Area"] == 'Belgium-Luxembourg']

,Area,Year,Value
611,Belgium-Luxembourg,1970,497.72
612,Belgium-Luxembourg,1971,509.71
613,Belgium-Luxembourg,1972,511.82
614,Belgium-Luxembourg,1973,528.93
615,Belgium-Luxembourg,1974,520.95
616,Belgium-Luxembourg,1975,451.44
617,Belgium-Luxembourg,1976,463.11
618,Belgium-Luxembourg,1977,457.56
619,Belgium-Luxembourg,1978,477.81
620,Belgium-Luxembourg,1979,455.68


In [ ]:
df_f_subset = df_f_subset.loc[~df_f_subset['Area'].isin(["China, Taiwan Province of","China, mainland"])]
df_f_subset

,Area,Year,Value
0,Afghanistan,1970,2.42
1,Afghanistan,1971,2.55
2,Afghanistan,1972,3.62
3,Afghanistan,1973,3.06
4,Afghanistan,1974,4.21
...,...,...,...
8275,Zimbabwe,2016,31.78
8276,Zimbabwe,2017,35.71
8277,Zimbabwe,2018,32.39
8278,Zimbabwe,2019,32.39


In [ ]:
#sample size of final sample
len(list(df_f_subset["Area"].unique()))

188

In [ ]:
df_f_subset["Area"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Aruba', 'Australia',
       'Austria', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados',
       'Belgium-Luxembourg', 'Belize', 'Benin', 'Bermuda', 'Bhutan',
       'Bolivia (Plurinational State of)', 'Botswana', 'Brazil',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Chile', 'China',
       'China, Hong Kong SAR', 'Colombia', 'Comoros', 'Congo',
       'Cook Islands', 'Costa Rica', 'Cuba', 'Cyprus', 'Czechoslovakia',
       "Côte d'Ivoire", "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Eswatini',
       'Ethiopia PDR', 'Fiji', 'Finland', 'France', 'French Guiana',

In [ ]:
#cleaning for missing values in potato dataset
df_f_final = df_f_subset.loc[df_f_subset['Area'].isin(['Afghanistan','Albania','Algeria','Angola','Argentina','Australia','Austria','Bangladesh','Belgium-Luxembourg','Bhutan','Bolivia (Plurinational State of)','Brazil','Bulgaria','Burundi','Cabo Verde','Cameroon','Canada','Central African Republic','Chad','Chile','China','China, Taiwan Province of','China, mainland','Colombia','Congo','Costa Rica','Cuba','Cyprus','Czechoslovakia',"Democratic People's Republic of Korea",'Democratic Republic of the Congo','Denmark','Dominica','Dominican Republic','Ecuador','Egypt','El Salvador','Eswatini','Ethiopia PDR','Faroe Islands','Fiji','Finland','France','French Polynesia','Germany','Greece','Guatemala','Haiti','Honduras','Hungary','Iceland','India','Indonesia','Iran (Islamic Republic of)','Iraq','Ireland','Israel','Italy','Jamaica','Japan','Jordan','Kenya',"Lao People's Democratic Republic",'Lebanon','Lesotho','Libya','Madagascar','Malawi','Malta','Mauritania','Mauritius','Mexico','Mongolia','Morocco','Mozambique','Myanmar','Nepal','Netherlands (Kingdom of the)','New Caledonia','New Zealand','Nicaragua','Nigeria','Norway','Pakistan','Panama','Papua New Guinea','Paraguay','Peru','Philippines','Poland','Portugal','Republic of Korea','Romania','Rwanda','Saint Kitts and Nevis','Saudi Arabia','Senegal','South Africa','Spain','Sri Lanka','Sweden','Switzerland','Syrian Arab Republic','Thailand','Timor-Leste','Tunisia','Turkey','USSR','Uganda','United Kingdom of Great Britain and Northern Ireland','United Republic of Tanzania','United States of America','Uruguay','Venezuela (Bolivarian Republic of)','Viet Nam','Yemen','Yugoslav SFR','Zambia','Zimbabwe'])]

In [ ]:
#sample size of final sample
len(list(df_f_final["Area"].unique()))

116

In [ ]:
df_f_final.loc[df_f_final["Area"] == 'USSR']

,Area,Year,Value
7567,USSR,1970,44.260
7568,USSR,1971,48.740
7569,USSR,1972,53.140
7570,USSR,1973,57.940
7571,USSR,1974,63.290
7572,USSR,1975,74.190
7573,USSR,1976,76.270
7574,USSR,1977,77.500
7575,USSR,1978,79.360
7576,USSR,1979,75.010


In [ ]:
df_f_final["Area"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
       'Australia', 'Austria', 'Bangladesh', 'Belgium-Luxembourg',
       'Bhutan', 'Bolivia (Plurinational State of)', 'Brazil', 'Bulgaria',
       'Burundi', 'Cabo Verde', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo', 'Costa Rica', 'Cuba', 'Cyprus', 'Czechoslovakia',
       "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Eswatini', 'Ethiopia PDR', 'Fiji', 'Finland', 'France',
       'French Polynesia', 'Germany', 'Greece', 'Guatemala', 'Haiti',
       'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia',
       'Iran (Islamic Republic of)', 'Iraq', 'Ireland', 'Israel', 'Italy',
       'Jamaica', 'Japan', 'Jordan', 'Kenya',
       "Lao People's Democratic Republic", 'Lebanon', 'Lesotho', 'Libya',
       'Madagascar', 'Malawi

In [ ]:
#export dataset

df_f_final.to_csv("Fertilizer.csv", index=False)